<a href="https://colab.research.google.com/github/HenryLiu714/BackTester/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing and Importing Dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
# Ensure dependencies are installed

dependencies = [
    "numpy",
    "pandas",
    "matplotlib",
    "tweetnlp"
]

for dependency in dependencies:
  os.system(f"pip3 install {dependency}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tweetnlp

In [ ]:
os.chdir("drive/MyDrive/sentiment_bot")

**Sentiment Analysis Model Loading**

In [ ]:
model = tweetnlp.load_model("sentiment")

In [ ]:
def get_sentiment(s: str) -> int:
  sentiment = model.sentiment(s)['label']

  if sentiment == 'positive':
    return 1
  elif sentiment == 'negative':
    return -1
  else:
    return 0

In [ ]:
tweets = pd.read_csv('stock.csv', on_bad_lines='skip')

In [ ]:
tweets = tweets.dropna()
tweets = tweets[tweets['category']=='stock_images']
tweets = tweets.drop(columns=['tweet_url', 'tweet_type', 'price_of_ticker', 'change_of_ticker', 'category'])

In [ ]:
def tickers_to_array(s:str)->list[str]:
  s = s[1:-1].split(', ')
  tickers = []
  for ticker in s:
    tickers.append(ticker[2:-1])
  return tickers

tweets['tickers_mentioned'] = tweets['tickers_mentioned'].transform(lambda x: tickers_to_array(x))
tweets['timestamp'] = tweets['timestamp'].transform(lambda x: pd.Timestamp(x[:10]))

In [ ]:
class Trade:
  def __init__(self, ticker: str, dir: bool, strength: int):
    self.ticker = ticker
    self.dir = dir
    self.strength = strength

In [ ]:
def get_best_trade(date: pd.Timestamp) -> Trade:
  ticker_data = tweets[tweets['timestamp'] == date]

  tickers = {}

  for index, row in ticker_data.iterrows():
    sentiment = get_sentiment(row['tweet_text'])

    for ticker in row['tickers_mentioned']:
      if ticker not in tickers:
        tickers[ticker] = sentiment
      else:
        tickers[ticker] += sentiment

  best_ticker = max(tickers, key=tickers.get)
  worst_ticker = min(tickers, key=tickers.get)

  if abs(tickers[best_ticker]) > abs(tickers[worst_ticker]):
    return Trade(best_ticker, True, abs(tickers[best_ticker]))
  else:
    return Trade(worst_ticker, False, abs(tickers[worst_ticker]))
